In [2]:
import numpy as np
import pandas as pd

In [3]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [4]:
movies = movies.merge(credits, on='title')

In [5]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [6]:
movies.dropna(inplace=True)
movies.iloc[0, 3]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [7]:
def convert(genre):
    st = 0
    idx = genre.find('"name": "', st)
    lst = []
    while idx != -1:
        st = idx + 9
        lst.append(str(genre[st:genre.find('"', st)]))
        idx = genre.find('"name": "', st)
    return lst

def Convert(genre):
    st = 0
    idx = genre.find('"name": "', st)
    lst = []
    l = 3
    while idx != -1 and l > 0:
        st = idx + 9
        lst.append(str(genre[st:genre.find('"', st)]))
        idx = genre.find('"name": "', st)
        l -= 1
    return lst

from ast import literal_eval
def conv(obj):
    for i in literal_eval(obj):
        if i['job'] == 'Director':
            return [i['name']]
    return []


In [8]:
movies['crew'] = movies['crew'].apply(conv).apply(lambda x: [i.replace(" ", "") for i in x])
movies['genres'] = movies['genres'].apply(convert).apply(lambda x: [i.replace(" ", "") for i in x])
movies['keywords'] = movies['keywords'].apply(convert).apply(lambda x: [i.replace(" ", "") for i in x])
movies['cast'] = movies['cast'].apply(Convert).apply(lambda x: [i.replace(" ", "") for i in x])
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [9]:
movies['tag'] = movies['overview'] + movies['genres'] + movies['cast'] + movies['keywords'] + movies['crew']

In [10]:
df = movies[['movie_id', 'title', 'tag']]
df['tag'] = df['tag'].apply(lambda x: ' '.join(x).lower())

C:\Users\rohan\AppData\Local\Temp\ipykernel_9096\2808198981.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tag'] = df['tag'].apply(lambda x: ' '.join(x).lower())


In [11]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words='english')

In [12]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [13]:
def stem(text):
    stemmed = []
    for i in text.split():
        stemmed.append(ps.stem(i))
    return ' '.join(stemmed)

In [14]:
df['tag'] = df['tag'].apply(stem)
df

C:\Users\rohan\AppData\Local\Temp\ipykernel_9096\4014118289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tag'] = df['tag'].apply(stem)


,movie_id,title,tag
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just want to play hi guitar and ca...
4805,72766,Newlyweds,a newlyw couple' honeymoon is upend by the arr...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduc a dedic q..."
4807,126186,Shanghai Calling,when ambiti new york attorney sam is sent to s...


In [15]:
vector = cv.fit_transform(df['tag']).toarray()

In [16]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [17]:
def recommend(movieName):
    movieName = movieName.strip().lower()
    idx = list(df[df['title'] == movieName].index)
    if len(idx) == 0:
        return "No such movie!"
    sims = sorted(list(enumerate(similarity[idx[0]])), key=lambda x: x[1], reverse=True)[1:6]
    return [df.iloc[i[0], 1] for i in sims]

print(recommend('thor'))

No such movie!


In [18]:
movieDict = {}
Id = {}
for i in range(df.shape[0]):
    Id[i] = int(df.iloc[i, 0])
for i in range(df.shape[0]):
    movieDict[df.iloc[i, 1]] = [Id[x[0]] for x in sorted( list(enumerate(similarity[i])), reverse=True, key=lambda y: y[1])[:6]]
movieDict

{'Avatar': [19995, 440, 679, 270938, 602, 7450],
 "Pirates of the Caribbean: At World's End": [285, 58, 22, 1865, 87827, 173],
 'Spectre': [206647, 10764, 37724, 36670, 657, 700],
 'The Dark Knight Rises': [49026, 155, 364, 2661, 414, 272],
 'John Carter': [49529, 87421, 32740, 16651, 188207, 10461],
 'Spider-Man 3': [559, 558, 557, 102382, 1930, 6488],
 'Tangled': [38757, 244339, 10198, 13700, 50135, 10193],
 'Avengers: Age of Ultron': [99861, 68721, 10138, 1726, 10195, 24428],
 'Harry Potter and the Half-Blood Prince': [767, 675, 674, 672, 671, 673],
 'Batman v Superman: Dawn of Justice': [209112,
  49521,
  272,
  10195,
  13183,
  34769],
 'Superman Returns': [1452, 8536, 9531, 11411, 1924, 76170],
 'Quantum of Solace': [10764, 709, 206647, 681, 36557, 36670],
 "Pirates of the Caribbean: Dead Man's Chest": [58,
  285,
  1865,
  22,
  72197,
  11128],
 'The Lone Ranger': [57201, 38665, 576, 10718, 68818, 1725],
 'Man of Steel': [49521, 1452, 209112, 8536, 49538, 76757],
 'The Chroni

In [19]:
import pickle

with open('data.pkl', 'wb') as file:
    pickle.dump(movieDict, file)